In [1]:
# !pip install langchain
# !pip install langchain-experimental
# !pip install langchain-groq
# !pip install langchain-google-genai

In [ ]:
# Importing the libraries
import os
import pandas as pd
import numpy as np
import langchain
from langchain_groq import ChatGroq
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.prompts import PromptTemplate
import re
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
import sys
import ast
import pandas as pd

# IMPORTING DATASET

In [9]:
df=pd.read_csv('ffinal_Emarketing.csv')

In [10]:
df.head()

,Unnamed: 0,Shop_Name,Type,Address,State,Zip Code,Phone,Website,Rating,Reviews,Hours,Operating Hours,Latitude,Longitude
0,0,Dallas Wholesale & Supply,Wholesaler,"10228 Northwest Hwy #1117, Dallas, TX 75238",TX,75238,NaN,http://dallaswholesalecompany.com/,1.0,1.0,Open ⋅ Closes 4 PM,"{'thursday': '8\u202fAM–4\u202fPM', 'friday': ...",32.863813,-96.709531
1,1,Los Rancheros Mexican Restaurant,Bar,"10261 Ferguson Rd, Dallas, TX 75228",TX,75228,(214) 238-3583,NaN,4.2,1463.0,Open ⋅ Closes 10 PM,"{'thursday': '11\u202fAM–10\u202fPM', 'friday'...",32.833952,-96.674281
2,2,Five 5 Smokinos,Tobacco shop,"10325 Lake June Rd ste 580, Dallas, TX 75217",TX,75217,(945) 386-2222,https://www.facebook.com/profile.php?id=100088...,4.2,11.0,Open ⋅ Closes 10 PM,"{'thursday': '10\u202fAM–10\u202fPM', 'friday'...",32.735040,-96.645523
3,3,SMOKE HUB DISTRO- VAPES AND SMOKE WHOLESALE & ...,Wholesaler,"10430 Shady Trail #102, Dallas, TX 75220",TX,75220,(214) 903-9226,http://smokehubdistro.com/,4.9,23.0,Open ⋅ Closes 6 PM,"{'thursday': '9\u202fAM–6\u202fPM', 'friday': ...",32.865641,-96.889162
4,4,Prime Supply Distro,Distribution service,"10430 Shady Trail Ste 108, Dallas, TX 75220",TX,75220,NaN,http://primesupplydistro.com/,5.0,2.0,Open ⋅ Closes 5 PM,"{'thursday': '10\u202fAM–5\u202fPM', 'friday':...",32.864783,-96.888557


In [11]:
df['Type'].unique()

array(['Wholesaler', 'Bar', 'Tobacco shop', 'Distribution service',
       'Smoke shop', 'Grocery store', 'Vaporizer store',
       'Convenience store', 'Hookah store', 'Wholesale grocer',
       'Department store', 'Produce wholesaler', 'Restaurant',
       'American restaurant', 'Novelties wholesaler', 'Supermarket',
       'Sports bar', 'Cigar shop', 'Food products supplier', 'Irish pub',
       'Store', 'Florist', 'Wholesale market', 'Bar & grill',
       'Cocktail bar', 'Lounge', 'Tiki bar', 'Pub', 'Wholesale florist',
       'General store', 'Beer distributor',
       'Party equipment rental service', 'Variety store',
       'Marine supply store', 'Karaoke bar', 'Liquor store', 'Warehouse',
       'Seafood restaurant', 'Mexican restaurant', 'Gift shop',
       'Western apparel store', 'Live music bar', 'Tapas bar',
       'Manufacturer', 'Hookah bar'], dtype=object)

# filtering the dataframe

In [42]:
categories = ['Tobacco shop', 'Vaporizer store', 'Smoke shop','Cigar shop']

filtered_df = df[df['Type'].isin(categories)]

new_df=filtered_df[['Shop_Name','Type','Address','State','Zip Code']]

In [44]:
new_df.head()

,Shop_Name,Type,Address,State,Zip Code
2,Five 5 Smokinos,Tobacco shop,"10325 Lake June Rd ste 580, Dallas, TX 75217",TX,75217
6,Smoke N Peace,Smoke shop,"10531 E NW Hwy ste d, Dallas, TX 75238",TX,75238
9,RaceTrac,Tobacco shop,"10707 Ferguson Rd, Dallas, TX 75228",TX,75228
10,NOVELTY VAPE & MORE,Vaporizer store,"10710 Shiloh Rd, Dallas, TX 75228",TX,75228
17,Smoke Depot Wholesale/Retail,Vaporizer store,"11029 Harry Hines Blvd C-105, Dallas, TX 75229",TX,75229


# making instance of llm

In [45]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.2, google_api_key="your_api_key")

In [59]:
data_dict = []
# Shop_Name	Type	Address	State	Zip Code
for index, row in new_df.iterrows():
    customer_dict = {
        'Shop_Name': row['Shop_Name'],
        'Type': row['Type'],
        'Address': row['Address'],
        'State': row['State'],
        'Zip Code': row['Zip Code'],

    }

    data_dict.append(customer_dict)

print(data_dict)


[{'Shop_Name': 'Five 5 Smokinos', 'Type': 'Tobacco shop', 'Address': '10325 Lake June Rd ste 580, Dallas, TX 75217', 'State': 'TX', 'Zip Code': 75217}, {'Shop_Name': 'Smoke N Peace', 'Type': 'Smoke shop', 'Address': '10531 E NW Hwy ste d, Dallas, TX 75238', 'State': 'TX', 'Zip Code': 75238}, {'Shop_Name': 'RaceTrac', 'Type': 'Tobacco shop', 'Address': '10707 Ferguson Rd, Dallas, TX 75228', 'State': 'TX', 'Zip Code': 75228}, {'Shop_Name': 'NOVELTY VAPE & MORE', 'Type': 'Vaporizer store', 'Address': '10710 Shiloh Rd, Dallas, TX 75228', 'State': 'TX', 'Zip Code': 75228}, {'Shop_Name': 'Smoke Depot Wholesale/Retail', 'Type': 'Vaporizer store', 'Address': '11029 Harry Hines Blvd C-105, Dallas, TX 75229', 'State': 'TX', 'Zip Code': 75229}, {'Shop_Name': 'NOVELTY WHOLESALE SMOKES AND VAPOR', 'Type': 'Smoke shop', 'Address': '11076 Harry Hines Blvd # 100, Dallas, TX 75229', 'State': 'TX', 'Zip Code': 75229}, {'Shop_Name': 'FUBU Smoke N Vapes', 'Type': 'Vaporizer store', 'Address': '11090 Estat

### BELOW CODE IS DONE FOR CONVERTING SHOPS INTO JSNO FORMAT

In [60]:
j=0
for i in data_dict:
    j+=1
    print(i)
    print("\n")

print("There are ",j," items in the list")

{'Shop_Name': 'Five 5 Smokinos', 'Type': 'Tobacco shop', 'Address': '10325 Lake June Rd ste 580, Dallas, TX 75217', 'State': 'TX', 'Zip Code': 75217}


{'Shop_Name': 'Smoke N Peace', 'Type': 'Smoke shop', 'Address': '10531 E NW Hwy ste d, Dallas, TX 75238', 'State': 'TX', 'Zip Code': 75238}


{'Shop_Name': 'RaceTrac', 'Type': 'Tobacco shop', 'Address': '10707 Ferguson Rd, Dallas, TX 75228', 'State': 'TX', 'Zip Code': 75228}


{'Shop_Name': 'NOVELTY VAPE & MORE', 'Type': 'Vaporizer store', 'Address': '10710 Shiloh Rd, Dallas, TX 75228', 'State': 'TX', 'Zip Code': 75228}


{'Shop_Name': 'Smoke Depot Wholesale/Retail', 'Type': 'Vaporizer store', 'Address': '11029 Harry Hines Blvd C-105, Dallas, TX 75229', 'State': 'TX', 'Zip Code': 75229}


{'Shop_Name': 'NOVELTY WHOLESALE SMOKES AND VAPOR', 'Type': 'Smoke shop', 'Address': '11076 Harry Hines Blvd # 100, Dallas, TX 75229', 'State': 'TX', 'Zip Code': 75229}


{'Shop_Name': 'FUBU Smoke N Vapes', 'Type': 'Vaporizer store', 'Address': '11090 

### the below code is the prompt template which we pass into prompt in llm

In [63]:
li_formatted_email=[]

def mail_format(data_dict):

        j=0

        for i in data_dict:

            data=i

            mail_format = """

      Subject: Enhance Your Inventory with NEPA WHOLESALE's Vape & Tobacco Products

      Dear {Shop_Name} Team,

      I hope this message finds you well. I'm reaching out from NEPA WHOLESALE LLC, a leading distributor of high-quality vape and tobacco related products.

      We specialize in providing a comprehensive range of products to meet diverse customer preferences. Our offerings include:

      - Extensive selection of vape devices and e-liquids
      - Premium tobacco products
      - Accessories and replacement parts
      - Emerging brands and innovative items

      We also provide services to hookah bars, gas stations, general stores, and more.

      If you're interested in exploring how we can support your business needs, I'd be happy to provide more information or arrange a brief discussion.

      For any questions or to request our latest product catalog, please don't hesitate to contact me at [our Phone Number] or reply to this email.

      Thank you for your time and consideration.

      Best regards,

      NEPA WHOLESALE LLC

      Visit our website: [https://nepa2wholesale.com/]
      """

            # Fill the placeholders in the email template
            formatted_email = mail_format.format(

                Shop_Name=data['Shop_Name']

            )

            li_formatted_email.append(formatted_email)

            j+=1
            print("_"*150)

            print("Mail Format: ",j)
            print("_"*150)

            print(formatted_email)

            print("\n")


In [64]:
mail_format(data_dict)

Streaming output truncated to the last 5000 lines.


______________________________________________________________________________________________________________________________________________________
Mail Format:  251
______________________________________________________________________________________________________________________________________________________


      Subject: Enhance Your Inventory with NEPA WHOLESALE's Vape & Tobacco Products

      Dear VAPE & SMOKE RIVERSIDE MINI MART Team,

      I hope this message finds you well. I'm reaching out from NEPA WHOLESALE LLC, a leading distributor of high-quality vape and tobacco related products.

      We specialize in providing a comprehensive range of products to meet diverse customer preferences. Our offerings include:

      - Extensive selection of vape devices and e-liquids
      - Premium tobacco products
      - Accessories and replacement parts
      - Emerging brands and innovative items

      We also provide s

### below code is the prompt for llm

In [67]:

def Give_Email(femail):


    final_prompt = f"""

    You are an expert in paraphrasing emails. Your task is to refine and enhance the provided email to make it compelling and clear.
    The goal is to impress other shop owners and encourage them to use our products, ultimately attracting them as our customers. Each email should be unique, avoiding repetition, and written in a natural, conversational tone that mimics human speech patterns.

    Important: Don't provide more complex words and meaning while paraphasing, also Paraphrase the original email naturally while retaining these details.

    Here is the email to paraphrase:

    {femail}
    """



    final_prompt = PromptTemplate.from_template(template=final_prompt)

     # Define the LLM chain
    llm_chain = LLMChain(
                llm=llm,
                prompt=final_prompt
            )

     # Run the LLM chain with the formatted prompt
    refined_email = llm_chain.invoke({'email': femail})
    email_content = refined_email['text']


    return email_content

### below code is to generate the emails in the loop through the llm

In [68]:
final_emails = []
j=0
tot=len(li_formatted_email)
for i in li_formatted_email:
    j+=1
    tot-=1
    mail_to_send = Give_Email(i)
    file_path = 'mail_to_send.txt'

    try:
        with open(file_path, 'a') as file:
            data_str = str(mail_to_send)
            file.write("[")
            file.write(data_str + ',')
            file.write("],\n\n\n")

            print(f"{j} Mail Created |......................................| {tot} Mail Left")
    except Exception as e:
        print(f"Error: {e}")

    final_emails.append(mail_to_send)

# Print the entire list of emails in the specified format
emails_str = ','.join(final_emails)
print(f"[{emails_str}]")


Streaming output truncated to the last 5000 lines.

We're not just for smoke shops! We also cater to hookah bars, gas stations, and general stores.

If you're looking to upgrade your inventory and keep your customers coming back for more, let's chat. I'd love to show you how we can help you succeed.

Feel free to give me a call at [our Phone Number] or shoot me an email. You can also check out our website at [https://nepa2wholesale.com/] for our latest catalog.

Thanks for your time!

Best,

NEPA WHOLESALE LLC,**Paraphrased Email:**

**Subject: Elevate Your Shop with NEPA WHOLESALE's Vape and Tobacco Haven**

Hey there, Discount Tobacco & Grocery Team!

I'm writing from NEPA WHOLESALE, your go-to source for everything vape and tobacco. We've got you covered with a massive selection of:

* Top-notch vape devices and e-liquids
* Premium tobacco products
* Essential accessories and parts
* Exciting new brands and products

We're not just for vape shops! We've got what you need for hookah 

# new dataframe with emails

In [83]:

df = pd.DataFrame({
    'shop_name': new_df['Shop_Name'],
    'email_content': final_emails
})

In [71]:
df['email_content']

2      **Paraphrased Email:**\n\n**Subject: Elevate Y...
6      **Paraphrased Email:**\n\n**Subject: Elevate Y...
9      **Paraphrased Email:**\n\n**Subject: Elevate Y...
10     **Paraphrased Email:**\n\n**Subject: Elevate Y...
17     **Paraphrased Email:**\n\nSubject: Elevate You...
                             ...                        
735    **Paraphrased Email:**\n\nSubject: Elevate You...
736    **Paraphrased Email:**\n\n**Subject: Elevate Y...
737    **Paraphrased Email:**\n\n**Subject: Elevate Y...
746    **Paraphrased Email:**\n\n**Subject: Elevate Y...
747    **Paraphrased Email:**\n\n**Subject: Elevate Y...
Name: email_content, Length: 397, dtype: object

# preprocessing the generated emails

In [85]:
def remove_paraphrased(email):
    if isinstance(email, str):
        email = re.sub(r'\*\*Paraphrased Email:\*\*\n\n', '', email)
    return email

In [86]:
df['email_content'] = df['email_content'].apply(remove_paraphrased)

In [87]:
def remove_star(email):
    if isinstance(email, str):
        return re.sub(r"^\*\*Subject:", "Subject:", email, flags=re.MULTILINE)
    return email

# Apply the function to each email in the email_content column
df['email_content'] = df['email_content'].apply(remove_star)

# printing the generated mails

In [88]:
for i in df['email_content']:
    print(i)


Subject: Elevate Your Shop with NEPA WHOLESALE's Vape and Tobacco Haven**

Hey Five 5 Smokinos Team,

Hope you're doing great! I'm writing from NEPA WHOLESALE, the go-to source for all things vape and tobacco.

We've got a wide range of stuff to keep your customers coming back for more:

* The latest and greatest vape devices and e-liquids
* Top-notch tobacco products
* All the accessories and parts you need
* Cool new brands and innovative products

We're not just for vape shops, either. We've got what you need for hookah bars, gas stations, and even general stores.

If you're looking to up your game, let's chat! I'd love to show you how we can help.

Shoot me a call at [our Phone Number] or reply to this email. You can also check out our website at [https://nepa2wholesale.com/] for more info.

Thanks for your time!

Cheers,
NEPA WHOLESALE LLC
Subject: Elevate Your Shop with NEPA WHOLESALE's Vape & Tobacco Haven**

Hey there, Smoke N Peace Team!

I'm writing from NEPA WHOLESALE, the g

In [90]:
df.to_csv('customer_attract_mail.csv',index=False)